In [2]:
import os
import json
import re
import shutil
input_folder = "encoded"
output_folder = "done"
import torch
import torch.nn.functional as F
import textwrap

In [3]:
# Use list comprehension to create a list of file paths
verilog_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename.endswith('.txt')]

# Print the list of file paths
print(len(verilog_files))
print(type(verilog_files[0]))
print(verilog_files[0])


290
<class 'str'>
encoded\adder11.txt


In [23]:
with open("labels_dict.txt", 'r') as file:
    data = file.read()
data_dict = json.loads(data)
for key, value in data_dict.items():
    print(key, value)

adder 14
ALU 9
and 25
comparator 24
decoder 26
encoder 23
mult 24
mux 26
nand 20
nor 16
not 11
or 24
pe 16
sub 9
xnor 23


In [16]:
num_connections_list = []
size_list = []
num_nodes_files_list = []
num_edges_files_list = []
for file in verilog_files:
    with open(file, "r") as file:
        loaded_data = json.load(file)
        num_nodes_files_list.append(len(loaded_data[0]))
        num_edges_files_list.append(len(loaded_data[1][0]))
        for sublist in loaded_data[0]:
            if sublist[2] == 0:
                print(file.name)
                continue
            num_connections_list.append(sublist[2])
            size_list.append(sublist[3])

encoded\ALU10.txt
encoded\ALU13.txt
encoded\decoder30.txt
encoded\encoder16.txt
encoded\encoder18.txt
encoded\encoder6.txt
encoded\pe14.txt
encoded\pe15.txt
encoded\pe16.txt
encoded\pe22.txt
encoded\pe7.txt


In [17]:
average_connections = sum(num_connections_list) / len(num_connections_list)
max_connections = max(num_connections_list)
min_connections = min(num_connections_list)

print("Average connections: ", average_connections)
print("Max connections: ", max_connections)
print("Min connections: ", min_connections)

Average connections:  2.506062685884237
Max connections:  34
Min connections:  1


In [18]:
average_size = sum(size_list) / len(size_list)
max_size = max(size_list)
min_size = min(size_list)

print("Average node size: ", average_size)
print("Max node size: ", max_size)
print("Min node size: ", min_size)

Average node size:  2.578586135895676
Max node size:  32
Min node size:  1


In [19]:
avergae_num_nodes = sum(num_nodes_files_list) / len(num_nodes_files_list)
max_num_nodes = max(num_nodes_files_list)
min_num_nodes = min(num_nodes_files_list)

print("Average number of nodes: ", avergae_num_nodes)
print("Max number of nodes: ", max_num_nodes)
print("Min number of nodes: ", min_num_nodes)

Average number of nodes:  15.110344827586207
Max number of nodes:  165
Min number of nodes:  3


In [20]:
average_edges = sum(num_edges_files_list) / len(num_edges_files_list)
max_edges = max(num_edges_files_list)
min_edges = min(num_edges_files_list)

print("Average number of edges: ", average_edges)
print("Max number of edges: ", max_edges)
print("Min number of edges: ", min_edges)

Average number of edges:  18.575862068965517
Max number of edges:  208
Min number of edges:  2


In [40]:
sum_nodes = sum(num_nodes_files_list)
sum_nodes

4382

In [21]:
# write global statistics to file

In [137]:
files_dict = dict()
for key in data_dict.keys():
    num_nodes = 0
    num_edges = 0
    num_files = 0
    # node_sizes = []
    for file_path in verilog_files:
        st = file_path.split("\\")[1].split(".")[0]
        # print(st)
        pattern = f"^({key})\d+"
        # print(pattern)
        if re.match(pattern, st):
            # print("hi")
            num_files += 1
            with open(file_path, "r") as file:
                loaded_data = json.load(file)
                num_nodes += len(loaded_data[0])
                num_edges += len(loaded_data[1][0])
                # for sublist in loaded_data[0]:
                #     node_sizes.append(sublist[3])
                
    files_dict[key] = {
        "num_nodes": num_nodes,
        "num_edges": num_edges,
        "average_nodes": round(num_nodes / num_files,2),
        "average_edges": round(num_edges / num_files,2),
        "num_files": num_files,
        "percentage_nodes" : round(num_nodes / sum_nodes * 100,2),
        "percentage_edges" : round(num_edges / sum(num_edges_files_list) * 1000,2),
        "percentage_files" : round(num_files / len(verilog_files) * 100,2)
    }
# files_dict

In [138]:
files_dict["adder"]

{'num_nodes': 594,
 'num_edges': 877,
 'average_nodes': 42.43,
 'average_edges': 62.64,
 'num_files': 14,
 'percentage_nodes': 13.56,
 'percentage_edges': 162.8,
 'percentage_files': 4.83}

In [139]:
files_dict["ALU"]

{'num_nodes': 250,
 'num_edges': 378,
 'average_nodes': 27.78,
 'average_edges': 42.0,
 'num_files': 9,
 'percentage_nodes': 5.71,
 'percentage_edges': 70.17,
 'percentage_files': 3.1}

In [140]:
files_dict["and"]

{'num_nodes': 143,
 'num_edges': 142,
 'average_nodes': 5.72,
 'average_edges': 5.68,
 'num_files': 25,
 'percentage_nodes': 3.26,
 'percentage_edges': 26.36,
 'percentage_files': 8.62}

In [141]:
files_dict["comparator"]

{'num_nodes': 409,
 'num_edges': 502,
 'average_nodes': 17.04,
 'average_edges': 20.92,
 'num_files': 24,
 'percentage_nodes': 9.33,
 'percentage_edges': 93.19,
 'percentage_files': 8.28}

In [142]:
files_dict["decoder"]

{'num_nodes': 780,
 'num_edges': 1028,
 'average_nodes': 30.0,
 'average_edges': 39.54,
 'num_files': 26,
 'percentage_nodes': 17.8,
 'percentage_edges': 190.83,
 'percentage_files': 8.97}

In [143]:
files_dict["encoder"]

{'num_nodes': 400,
 'num_edges': 439,
 'average_nodes': 17.39,
 'average_edges': 19.09,
 'num_files': 23,
 'percentage_nodes': 9.13,
 'percentage_edges': 81.49,
 'percentage_files': 7.93}

In [144]:
files_dict["mult"]

{'num_nodes': 371,
 'num_edges': 463,
 'average_nodes': 15.46,
 'average_edges': 19.29,
 'num_files': 24,
 'percentage_nodes': 8.47,
 'percentage_edges': 85.95,
 'percentage_files': 8.28}

In [145]:
files_dict["mux"]

{'num_nodes': 419,
 'num_edges': 538,
 'average_nodes': 16.12,
 'average_edges': 20.69,
 'num_files': 26,
 'percentage_nodes': 9.56,
 'percentage_edges': 99.87,
 'percentage_files': 8.97}

In [146]:
files_dict["nand"]

{'num_nodes': 137,
 'num_edges': 135,
 'average_nodes': 6.85,
 'average_edges': 6.75,
 'num_files': 20,
 'percentage_nodes': 3.13,
 'percentage_edges': 25.06,
 'percentage_files': 6.9}

In [147]:
files_dict["nor"]

{'num_nodes': 120,
 'num_edges': 104,
 'average_nodes': 7.5,
 'average_edges': 6.5,
 'num_files': 16,
 'percentage_nodes': 2.74,
 'percentage_edges': 19.31,
 'percentage_files': 5.52}

In [148]:
files_dict["not"]

{'num_nodes': 57,
 'num_edges': 46,
 'average_nodes': 5.18,
 'average_edges': 4.18,
 'num_files': 11,
 'percentage_nodes': 1.3,
 'percentage_edges': 8.54,
 'percentage_files': 3.79}

In [149]:
files_dict["or"]

{'num_nodes': 147,
 'num_edges': 129,
 'average_nodes': 6.12,
 'average_edges': 5.38,
 'num_files': 24,
 'percentage_nodes': 3.35,
 'percentage_edges': 23.95,
 'percentage_files': 8.28}

In [150]:
files_dict["pe"]

{'num_nodes': 264,
 'num_edges': 317,
 'average_nodes': 16.5,
 'average_edges': 19.81,
 'num_files': 16,
 'percentage_nodes': 6.02,
 'percentage_edges': 58.85,
 'percentage_files': 5.52}

In [151]:
files_dict["sub"]

{'num_nodes': 131,
 'num_edges': 147,
 'average_nodes': 14.56,
 'average_edges': 16.33,
 'num_files': 9,
 'percentage_nodes': 2.99,
 'percentage_edges': 27.29,
 'percentage_files': 3.1}

In [152]:
files_dict["xnor"]

{'num_nodes': 160,
 'num_edges': 142,
 'average_nodes': 6.96,
 'average_edges': 6.17,
 'num_files': 23,
 'percentage_nodes': 3.65,
 'percentage_edges': 26.36,
 'percentage_files': 7.93}

In [155]:
nodes_sizes_dict = dict()
for key in data_dict.keys():
    node_sizes = []
    node_connections = []
    for file_path in verilog_files:
        st = file_path.split("\\")[1].split(".")[0]
        pattern = f"^({key})\d+"
        if re.match(pattern, st):
            num_files += 1
            with open(file_path, "r") as file:
                loaded_data = json.load(file)
                for sublist in loaded_data[0]:
                    node_sizes.append(sublist[3])
                    node_connections.append(sublist[2])
                
    nodes_sizes_dict[key] = {
        "node connections list": node_connections,
        "node size list" : node_sizes
    }
